# Understanding tensor

## Create tensors

In [1]:
import torch

tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1,2,3])
tensor2d = torch.tensor([[1,2],
                         [3,4]])
tensor3d = torch.tensor([[[1,2],[3,4]],
                         [[5,6],[7,8]]])

## Tensor data types

In [8]:
# Check type of tensor
tensor1d = torch.tensor([1,2,3])
print(tensor1d.dtype)

torch.int64


In [9]:
floatvec = torch.tensor([1.0,2.0,3.0])
print(floatvec.dtype)

torch.float32


In [10]:
# Convert int64-tensor to float32-tensor
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

torch.float32


## Tensor operations

In [11]:
tensor2d = torch.tensor([[1,2,3],
                         [4,5,6]])
print(tensor2d)

tensor([[1, 2, 3],
        [4, 5, 6]])


In [12]:
print(tensor2d.shape)

torch.Size([2, 3])


In [13]:
# Reshape tensor with <reshape>
print(tensor2d.reshape(3,2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [14]:
# Reshape tensor with <view>
print(tensor2d.view(3,2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [15]:
# Transpose
print(tensor2d.T)

tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [16]:
# Matrix multiplication (2 ways)
print(tensor2d.matmul(tensor2d.T))
print(tensor2d @ tensor2d.T)

tensor([[14, 32],
        [32, 77]])
tensor([[14, 32],
        [32, 77]])


# Seeing models as computation graphs

## Implement a logistic regression forward pass

In [17]:
import torch.nn.functional as F

# true label
y = torch.tensor([1.0]) 

x1 = torch.tensor([1.1]) # input feature
w1 = torch.tensor([2.2]) # weight parameter
b = torch.tensor([0.0]) # bias

# prediction
z = x1*w1+b 
a = torch.sigmoid(z) 

loss = F.binary_cross_entropy(a, y)

## Automatic differentiation

In [18]:
# Computing gradients via autograd
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])

x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True) # notice the flag
b = torch.tensor([0.0], requires_grad=True) # notice the flag

z = x1*w1+b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

# retain_graph: PyTorch will destroy graph when calling <grad> if we don't set this
grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [19]:
# PyTorch has already a built-in function to compute <grad>
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


# MLP

In [20]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
            
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            torch.nn.Linear(20, num_outputs)
        )
    def forward(self, x):
        logits = self.layers(x)
        return logits
        

In [21]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [22]:
# Count the number of trainable parameters 
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [23]:
# Check weight and bias of the 1st <Linear> layer
print(model.layers[0].weight)
print(model.layers[0].weight.shape)
print("\n")
print(model.layers[0].bias)
print(model.layers[0].bias.shape)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)
torch.Size([30, 50])


Parameter containing:
tensor([-0.1250,  0.0513,  0.0366,  0.0075,  0.0509,  0.0545, -0.0393,  0.0924,
        -0.1412, -0.1232, -0.1063,  0.0081, -0.1249,  0.0101, -0.0019, -0.1298,
         0.1388, -0.0330,  0.1017,  0.1247, -0.0554, -0.0417,  0.1388,  0.0159,
         0.1215,  0.0385,  0.0769, -0.1224, -0.0279,  0.0991],
       requires_grad=True)
torch.Size([30])


In [24]:
# Try a forward pass 
torch.manual_seed(123)
X = torch.rand((1,50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [25]:
# in INFERENCE we don't need to compute graph
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [26]:
# for prediction => <softmax> on logits
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


# Data loaders

In [27]:
# Create toy data
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0, 1])

In [28]:
# Create toy <Dataset>
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)
print(len(train_ds))

5


In [29]:
# Create toy <DataLoader> from <Dataset>
from torch.utils.data import DataLoader
torch.manual_seed(123)
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0 # number of background processes
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [30]:
# Walk through <train_loader>
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:\n", x, y)

Batch 1:
 tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2:
 tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3:
 tensor([[ 2.7000, -1.5000]]) tensor([1])


In [31]:
# Walk through another <train_loader> with <drop_last>
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:\n", x, y)

Batch 1:
 tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2:
 tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


# Training loop

In [32]:
import torch.nn.functional as F 

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(
    model.parameters(), 
    lr=0.5
)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad() 
        loss.backward() # compute gradients
        optimizer.step() # updates parameters

        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train Loss: {loss:.2f}")
    model.eval()
    # model evaluation code


Epoch: 001/003 | Batch 000/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.00


In [33]:
# number of params
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 752


In [34]:
# make prediction
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [35]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [36]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [37]:
# evaluate predictions - quick check
print(predictions == y_train)

tensor([True, True, True, True, True])


In [38]:
# evaluate predictions
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total_examples = 0 # number of samples; usually = number of samples in Dataset if we have enough memory
    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return (correct / total_examples).item()


In [39]:
# train accuracy
print(compute_accuracy(model, train_loader))

1.0


In [40]:
# test accuracy
print(compute_accuracy(model, test_loader))

1.0


# Saving and loading models

In [41]:
# Save
torch.save(model.state_dict(), "model.pth")

In [42]:
# Load
model = NeuralNetwork(2, 2)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

# Optimizing training performance with GPUs

In [43]:
# Check if GPU is available
print(torch.cuda.is_available())

True


In [44]:
# Carry on computation on CPU
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])
print(tensor_1 + tensor_2)

tensor([5., 7., 9.])


In [45]:
# Carry on computation on GPU
tensor_1 = tensor_1.to("cuda:0")
tensor_2 = tensor_2.to("cuda:0")
print(tensor_1 + tensor_2)

tensor([5., 7., 9.], device='cuda:0')


In [46]:
# # Error if not on same device
# tensor_1 = tensor_1.to("cpu")
# print(tensor_1 + tensor_2)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

## Training on single GPU 

In [48]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features,labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Epoch: {epoch+1:-3d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val loss: {loss:.2f}")
    model.eval()
    # Insert optional model evaluation code
    

Epoch:   1/003 | Batch 000/002 | Train/Val loss: 0.75
Epoch:   1/003 | Batch 001/002 | Train/Val loss: 0.65
Epoch:   2/003 | Batch 000/002 | Train/Val loss: 0.44
Epoch:   2/003 | Batch 001/002 | Train/Val loss: 0.13
Epoch:   3/003 | Batch 000/002 | Train/Val loss: 0.03
Epoch:   3/003 | Batch 001/002 | Train/Val loss: 0.00


## Training on multiple GPUs

Execute the <multiple-gpus.py> code, not here

## Exercise A.2

In [6]:
# Performance on CPU
a = torch.rand(100,200)
b = torch.rand(200,300)
%timeit a@b

36.3 μs ± 981 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
# Performance on GPU
a, b = a.to("cuda:0"), b.to("cuda:0")
%timeit a@b

18.1 μs ± 1.01 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
